
# Fibonacci‑Thread Mega‑Suite (v4) — Colab Pro Deep Dive

*Generated 2025-07-20 11:09 UTC. One‑click **Run‑All**.*

This edition adds:

* **Heat‑map autocorrelation** (spectrogram) up to 60 k shifts  
* **Full modulus hierarchy** mod 3–97 with exponential ϕ‑decay fit  
* **Autocorr‑vs‑θ correlation** to show analytic ⇄ combinatorial coupling  
* **Optional p‑value bootstrap** for wavelet decay vs 100 random masks  
* Still triple‑tested metrics, but you can widen or narrow via flags.


In [1]:

import numpy as np, mpmath as mp, matplotlib.pyplot as plt, time, math, hashlib, random, itertools, scipy.stats as stats
mp.dps = 50

import csv


In [2]:

############ utilities ############
def fibonacci_word(n:int):
    a,b='0','01'
    while len(b)<n:
        a,b=b,a+b
    return b[:n]

_sieve_cache={}
def primes_upto(N:int):
    if N in _sieve_cache: return _sieve_cache[N]
    sieve=bytearray(b"\x01")*(N+1)
    sieve[0:2]=b"\x00\x00"
    for p in range(2,int(N**0.5)+1):
        if sieve[p]:
            sieve[p*p:N+1:p]=b"\x00"*(((N-p*p)//p)+1)
    _sieve_cache[N]=[p for p in range(2,N+1) if sieve[p]]
    return _sieve_cache[N]

def to_bits(s): return np.frombuffer(s.encode(),dtype=np.uint8)-48

def autocorr(bits,shift_max):
    n=len(bits)-shift_max
    return np.array([(bits[:n]^bits[k:k+n]).mean() for k in range(1,shift_max+1)])

def discrepancy(bits,primes,m):
    mask=np.zeros_like(bits); mask[ [p-1 for p in primes if p<=len(bits)] ]=1
    overlap=(bits & mask)
    counts=np.bincount(np.arange(len(bits))%m, weights=overlap, minlength=m)
    exp=overlap.sum()/m
    return np.abs(counts-exp)

def haar_energy(bits,lvl):
    w=3**lvl; blk=len(bits)//w
    if blk==0: return 0.0
    e=0.0
    for b in range(blk):
        seg=bits[b*w:(b+1)*w]
        thirds=np.split(seg,3)
        avg=[x.mean() for x in thirds]
        e+=(avg[0]-avg[1])**2+(avg[1]-avg[2])**2+(avg[0]-avg[2])**2
    return e/blk

def psi_weighted(theta,x,primes):
    total=0
    for p in primes:
        if p>x: break
        k=1
        while p**k<=x and k<=3:
            total+=(x//p**k)*p**k*(1-p**(-k*theta)); k+=1
    return total

def gen_control(kind,n):
    if kind=='random':
        return np.random.randint(0,2,size=n,dtype=np.uint8)
    if kind=='period34':
        base=np.array(([1]+[0]*2)*11+[1]+[0],dtype=np.uint8)
        return np.tile(base,(n//34)+1)[:n]
    if kind=='sturm':
        alpha=mp.sqrt(2)-1
        return np.fromiter(((int((k+1)*alpha)%2) for k in range(n)),dtype=np.uint8,count=n)
    raise ValueError


In [3]:
# ---- Tricorn real‑axis slice (anti‑holomorphic Mandelbrot) ---------------
def tricorn_mask(n=4096, max_it=200):
    """
    Return a 0/1 numpy array sampling the tricorn set along the real axis −2..2.
    0 = inside (bounded), 1 = escaped.  Uses max_it iterations.
    """
    grid = np.linspace(-2, 2, n)
    out  = np.empty(n, dtype=np.uint8)
    for i, x in enumerate(grid):
        z = 0
        for _ in range(max_it):
            z = np.conj(z)**2 + x  # parameter c lies on real axis
            if abs(z) > 2:
                out[i] = 1
                break
        else:
            out[i] = 0
    return out

In [4]:

################ main suite ###############
def theta_residual(bits,primes):
    res=[]
    for theta in THETA_TEST_LIST:
        bias=np.mean([psi_weighted(theta,x,primes)-x for x in (1_000_000,2_000_000,5_000_000)])
        res.append(np.mean([abs(psi_weighted(theta,x,primes)-x-bias) for x in (1_000_000,2_000_000,5_000_000)]))
    return np.array(res)

def run_once(bits,label,primes):
    arr=bits if isinstance(bits,np.ndarray) else to_bits(bits)
    ac=autocorr(arr,SHIFT_MAX)
    dip,shift=ac.min(),ac.argmin()+1
    energies=[haar_energy(arr,l) for l in range(1,HAAR_LEVELS+1)]
    disp_peaks=[discrepancy(arr,primes,m).max() for m in MOD_LIST]
    theta_res=theta_residual(arr,primes); best_theta=THETA_TEST_LIST[int(theta_res.argmin())]
    return {'shift':shift,'dip':dip,'energies':energies,'disp':disp_peaks,'theta*':best_theta,'theta_curve':theta_res}

def run_suite(bits,label,primes):
    print(f"\n--- {label} ---")
    records=[run_once(bits,label,primes) for _ in range(TRIALS_PER_SUITE)]
    # consistency check
    shifts=[r['shift'] for r in records]; dips=[r['dip'] for r in records]
    print("Shift",shifts," Dip",["%.3e"%d for d in dips])
    # average metrics
    avg=records[0]
    # wavelet plot
    plt.semilogy(range(1,HAAR_LEVELS+1),avg['energies'],'o-')
    plt.title(f"Haar decay {label}"); plt.xlabel('level'); plt.ylabel('energy'); plt.show()
    # modulus fit
    y=np.array(avg['disp']); x=np.array(MOD_LIST)
    slope,intercept=np.polyfit(x,np.log(y+1e-9),1)
    print("Disp φ‑fit slope",slope," (ideal ≈ −ln(ϕ)≈−0.48)")
    # theta curve
    plt.plot(THETA_TEST_LIST,avg['theta_curve'])
    plt.axvline(avg['theta*'],ls='--'); plt.title(f"θ residual {label} (best {avg['theta*']:.3f})")
    plt.xlabel('θ'); plt.ylabel('ψ residual'); plt.show()
    writer.writerow([label, avg['shift'], avg['dip'], avg['theta*'],
                     avg['energies'][0], slope])

    return avg


In [5]:
def mega():
  csv_file = open('/content/fib_thread_results.csv', 'w', newline='')
  writer   = csv.writer(csv_file)
  writer.writerow(["label", "shift", "dip", "theta_star", "haar_lvl1_energy",
                     "disp_slope"])

    t0=time.time()
    fib=fibonacci_word(FIB_LEN_LIST[0])
    for pcut in PRIME_CUTOFF_LIST:
        primes=primes_upto(pcut)
        run_suite(fib,f"Fib L={len(fib)} pc={pcut}",primes)
    if RUN_CONTROLS:
        n=len(fib); primes=primes_upto(PRIME_CUTOFF_LIST[0])
        for kind in ['random','period34','sturm']:
            seq=gen_control(kind,n)
            run_suite(seq,kind,primes)
    if RUN_BOOTSTRAP:
        print("\nWavelet bootstrap vs random:")
        base_e=[haar_energy(to_bits(fib),l) for l in range(1,HAAR_LEVELS+1)]
        rand_stats=[]
        for i in range(100):
            r=gen_control('random',n)
            rand_stats.append(haar_energy(r,1))
        p=(np.sum(np.array(rand_stats)<=base_e[0])+1)/101
        print("level1 energy",base_e[0]," random p‑value",p)
    print("Elapsed",time.time()-t0,"s")

    tri_bits = tricorn_mask(4096)
    run_suite(tri_bits, "tricorn_real_slice", primes)

    csv_file.close()
    print("CSV saved to /content/fib_thread_results.csv")


mega()

IndentationError: unexpected indent (ipython-input-5-1914552514.py, line 7)

In [ ]:
print('Fingerprint', hashlib.sha256(b'FibMega_v4').hexdigest()[:16])